<a href="https://colab.research.google.com/github/Faraz-Khan02/Yes-Bank-Stock-Closing-Price-Prediction/blob/main/Yes_Bank_Stock_Closing_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - **Yes Bank Stock Closing Price Prediction**



##### **Project Type**    - Regression
##### **Contribution**    - Individual
##### **Name -** Faraz Faisal Khan


# **Project Summary -**

Write the summary here within 500-600 words.

# **GitHub Link -**

Provide your GitHub Link here.

# **Problem Statement**


**Yes Bank is a well-known bank in the Indian financial domain. Since 2018, it has been in the news because of the fraud case involving Rana Kapoor. Owing to this fact, it was interesting to see how that impacted the stock prices of the company and whether Time series models or any other predictive models can do justice to such situations. This dataset has monthly stock prices of the bank since its inception and includes closing, starting, highest, and lowest stock prices of every month. The main objective is to predict the stock’s closing price of the month.**

# ***Let's Begin !***

### Importing Libraries

In [ ]:
# Importing Libraries
import numpy as np
import pandas as pd
from numpy import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

### Dataset Loading

In [ ]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Loading Dataset
df = pd.read_csv("/content/drive/MyDrive/Capstone Project-2/data_YesBank_StockPrices.csv")

### Dataset First View

In [ ]:
# Dataset First Look
df.head()

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
df.shape

Our dataset contains 185 rows and 5 columns

### Dataset Information

In [ ]:
# Dataset Info
df.info()

Our dataset contains zero null values


Here we've the Yes Bank Stock price dataset which has the monthly stock prices. It contains the following features in it, they are:

**Date**: It denotes the month & year with respect to the price of the stock.

**Open**: It denotes the opening price at which a stock started trading that month.

**High**: It denotes the highest price at which a stock traded during a period.

**Low**: It denotes the highest price at which a stock traded during a period.

**Close**: It denotes the final trading price for that month.


#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
duplicate = df.duplicated()
print(duplicate.value_counts())

There is no duplicate value in our dataset.

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count
df.isnull().sum()


There is zero null value in our dataset.

In [ ]:
# Visualizing the missing values
plt.figure(figsize=(10, 4))
sns.heatmap(df.isnull(), cbar=True, yticklabels=False)
plt.xlabel("Column_name", size=12, weight="bold")
plt.title("Missing values",fontweight="bold",size=15)
plt.show()


It shows zero missing value.

## ***Data Cleaning***

In [ ]:
# Copying data to preserve orignal dataset
new_df = df.copy()

Here Date is not in regular format so we need to corvert it in 'YYYY-MM-DD' format.

In [ ]:
new_df['Date'] = new_df['Date'].apply(lambda x: datetime.strptime(x, "%b-%y"))

In [ ]:
new_df.head()

Here we changed our Date format. Now we are ready to do EDA on our dataset.

In [ ]:
# Segregating the dataset into dependent & independent variable.
X = new_df.drop(['Close','Date'],axis=1)       
Y = new_df['Close']                          

Here X is our Independent variable and Y is our Dependent variable.

## ***Exploratory Data Analysis***

### **Univariate Analysis**

In Univariate Analysis we will analize our Dependent variable and Independent variable separately using different Displot and bar plot.

In [ ]:

# Visualisation of closing price with respect to dates.
plt.figure(figsize=(15,5))
plt.xlabel('Year', fontsize=15)
plt.ylabel('Closing Prices', fontsize=15)
plt.plot(new_df['Date'], new_df['Close'],linewidth=2,color='red')
plt.title('Closing Stock Prices along different Year', fontsize=20)
plt.grid()
plt.show()


 After 2018 onwards the closing stock prices have witnessed a great downfall due to Rana Kapoor fraud case.

## *Distplot for Dependent Variable - Close*

In [ ]:
# Dependent variable-'Close'
plt.figure(figsize=(10,5))
sns.distplot(df['Close'],color="g")
plt.xlabel('Closing Price',fontsize=20)
plt.ylabel('Density',fontsize=20)
plt.show()

Here we can clearly see that distribution is right skewed so we have to do transformatiom to make it Normal using log transformation method.
Log Transfomation is used when our data is highly skewed.



In [ ]:
#Collecting all our numeric column in a new variable 
numeric_features=new_df.describe().columns
numeric_features
     

## *Mean and Median of Dependent Variable - Close*

In [ ]:
# Plotting Mean and Median of our Dependent variable
for col in numeric_features[-1:]:
    fig = plt.figure(figsize=(12,6))
    ax = fig.gca()
    feature = new_df[col]
    feature.hist(bins=50, ax = ax)
    ax.axvline(feature.mean(), color='red', linestyle='dashed', linewidth=2)
    ax.axvline(feature.median(), color='green', linestyle='dashed', linewidth=2)    
    ax.set_title(col)
plt.show()

Here red dashed line shows Mean of our target variable and green dashed line shows Median of our target variable.

Here there is lot of difference between our Mean and Median so we have to do Lof tranformation to reduce such a variance between our central tendency.

## *Applying Log Transformation on Dependent Variable - Close*

In [ ]:
# Applying log transformation on target variable 
plt.figure(figsize=(10,5))
sns.distplot(np.log(df['Close']),color="g")
plt.xlabel('Closing Price',fontsize=20)
plt.ylabel('Density',fontsize=20)
plt.show()

Now we can say that our data is transformed and it is normally distributed.

In [ ]:

# Applying log transformation on target variable
for col in numeric_features[-1:]:
    fig = plt.figure(figsize=(12,6))
    ax = fig.gca()
    feature = np.log(new_df[col])
    feature.hist(bins=50, ax = ax)
    ax.axvline(feature.mean(), color='red', linestyle='dashed', linewidth=2)
    ax.axvline(feature.median(), color='green', linestyle='dashed', linewidth=2)    
    ax.set_title(col)
plt.show()

After doing Log transformation the difference between Mean and Median is reduced to a great extent.

## *Distplot on Independent variable - (Open, High, Low)*

In [ ]:

# Distribution of independent variable
plt.figure(figsize=(10,5))
sns.distplot(new_df['Open'], color='purple')

plt.figure(figsize=(10,5))
sns.distplot(new_df['High'], color='brown')

plt.figure(figsize=(10,5))
sns.distplot(new_df['Low'], color='magenta')

plt.show()

Here just like our Dependent variable , all Independent variables are right skewed so, to convert it in normal distribution we need to do Log Transformation.

## *Mean & Median of Independent Variables - (Open, High, Low)*

In [ ]:
# Ploting bar blot for each independent variables - (Open, High, Low)
for col in numeric_features[:-1]:
    fig = plt.figure(figsize=(12, 6))
    ax = fig.gca()
    feature = new_df[col]
    feature.hist(bins=50, ax = ax)
    ax.axvline(feature.mean(), color='red', linestyle='dashed', linewidth=2)
    ax.axvline(feature.median(), color='green', linestyle='dashed', linewidth=2)    
    ax.set_title(col)
plt.show()

Here Red dashed line denotes Mean and Green dashed line denotes Median.

From above graph we can see that there is much difference in between mean and median for all independent variable. It means independent variables are not normally distributed.

## *Applying Log Transformation on Independent Variables - (Open, High, Low)*

In [ ]:

# Applying log transformation independent variables.
plt.figure(figsize=(10,5))
sns.distplot(np.log(new_df['Open']), color='purple')

plt.figure(figsize=(10,5))
sns.distplot(np.log(df['High']), color='brown')

plt.figure(figsize=(10,5))
sns.distplot(np.log(df['Low']), color='magenta')

plt.show()

Here we can see after applying Log Transformation out Independent variables are Normally Distributed.

In [ ]:
# Applying log transformation on Independent Variables
for col in numeric_features[:-1]:
    fig = plt.figure(figsize=(12, 6))
    ax = fig.gca()
    feature = np.log(new_df[col])
    feature.hist(bins=50, ax = ax)
    ax.axvline(feature.mean(), color='red', linestyle='dashed', linewidth=2)
    ax.axvline(feature.median(), color='green', linestyle='dashed', linewidth=2)    
    ax.set_title(col)
plt.show()

Here we can see after applying Log Transformation the difference between Mean & Median is almost finished.

## **Bivariate Analysis**

In Bivariate Analysis we will see relation between Dependent and Independent Variables using scatter plot and heat map.

## *Scatter plot for finding relation between Dependent & Independent variable*

In [ ]:
#Using scatter plot
for col in numeric_features[:-1]:
  fig = plt.figure(figsize = (10,5))
  ax = fig.gca()
  features = new_df[col]
  label = new_df['Close']
  correlation = features.corr(label)
  plt.scatter(x = features,y = label)
  plt.xlabel(col)
  plt.ylabel('Close')
  plt.title('Close Vs  ' + col + '_ correlation:' + str(correlation))
  z = np.polyfit(df[col],df['Close'],1)
  y_hat = np.poly1d(z)(df[col])
  plt.plot(df[col] , y_hat, "r--",lw = 2)
plt.show()

From above Scatter plot we can say all independent variables high correlation with our dependent variable.

So, we cannot drop any column for modelling all the independent variables are important.

## *Heatmap for checking correlation with Independent variables*

In [ ]:

#Correlation heatmap
plt.figure(figsize=(12,7))
sns.heatmap(new_df.corr(),cmap='PiYG',annot=True)

Here dependent variable shows high correlation with all independent variables.

## *Checking Multicollinearity*

In [ ]:
#Calculating VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calculate_vif(X):

  # calculating VIF
  vif =pd.DataFrame()
  vif["variables"] = X.columns
  vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X. shape[1])]

  return(vif)

calculate_vif(new_df[[i for i in new_df.describe().columns if i not in ['Date', 'Close']]])

Here VIF of every independent variable is high so we cannot drop any of the columns.

## *Train Test Split*

In [ ]:
# Data transformation
from scipy.stats import zscore
X = X.apply(zscore)
Y = np.log10(Y)

In [ ]:
#Creating Trainig and Testing set of our dataset
X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)
print('Shape of X_train Set:',X_train.shape)
print('Shape of X_test Set:',X_test.shape)
print('Shape of Y_train Set:',Y_train.shape)
print('Shape of Y_test Set:',Y_test.shape)

Here train_test_split() is used to split the dataset into training dataset and testing dataset.



*   **Train dataset**: It is the dataset which is used in model building. Here it constitutes of 80% of the dataset.
*   **Test dataset**: It is the dataset which is used to test our fit model.
Here it constitutes 20% of the dataset.



In [ ]:
# Transforming data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

MinMax Scaler shrinks the data within the given range, usually of 0 to 1. It transforms data by scaling features to a given range. It scales the values to a specific value range without changing the shape of the original distribution.

# **Linear Regression**

Linear Regression is a machine learning algorithm based on supervised learning. It performs a regression task. Regression models a target prediction value based on independent variables.Linear regression performs the task to predict a dependent variable value (y) based on a given independent variable (x). So, this regression technique finds out a linear relationship between x (input) and y(output).

In [ ]:
#Applying Linear Regression
regressor=LinearRegression()
regressor_model=regressor.fit(X_train,Y_train)

In [ ]:
#Checking the training accuracy of model
round(regressor.score(X_train,Y_train),4)

Here, our model training accuracy is 80.91%.

In [ ]:
# Test accuracy of linear regression model
round(regressor.score(X_test, Y_test), 4)

Here, our model test accuracy is 82.83%.

In [ ]:
#prediction of test data
Y_pred = regressor.predict(X_test)
Y_pred

In [ ]:
#converting Y_test data into array
np.array(Y_test)

In [ ]:
# Creating the DataFrame of test and train Dataset
train_Dataset = pd.DataFrame(X_train,Y_train)
test_Dataset = pd.DataFrame(Y_test)
test_Dataset.rename(columns= {'Close' :'Actual Closing Price'}, inplace =True)
     

In [ ]:
# Actual Closing Price VS Predicted closing Price
test_Dataset['Predicted Closing Price']= Y_pred
test_Dataset.head()

In [ ]:
# Test Performance of Linear Regregression
print("MSE :", round(mean_squared_error(Y_test, Y_pred), 4))
print("RMSE :", round(math.sqrt(mean_squared_error(Y_test, Y_pred)), 4)) 
print("MAE :", round(mean_absolute_error(Y_test, Y_pred), 4))
print("MAPE :", round(mean_absolute_percentage_error(Y_test, Y_pred), 4))
print("r2 :", round(r2_score(Y_test, Y_pred),4))



*  **MSE** : The Mean Squared Error of an estimator measures the average of error squares i.e. the average squared difference between the estimated values and true value. It is a risk function, corresponding to the expected value of the squared error loss. It is always non – negative and values close to zero are better.
*   **RMSE** :Root Mean Square Error is the standard deviation of the residuals (prediction errors). Residuals are a measure of how far from the regression line data points are; RMSE is a measure of how spread out these residuals are. In other words, it tells you how concentrated the data is around the line of best fit.
*   **MAE** : Mean Absolute Error calculates the average difference between the calculated values and actual values. It is also known as scale-dependent accuracy as it calculates error in observations taken on the same scale.
*   **MAPE** : It measures accuracy of a forecast system. It measures this accuracy as a percentage, and can be calculated as the average absolute percent error for each time period minus actual values divided by actual values.
*   **r2** : R-Squared is a statistical measure in a regression model that determines the proportion of variance in the dependent variable that can be explained by the independent variable.







In [ ]:
#Actual Price VS Predicted price for Linear Regression plot
plt.figure(figsize=(12,5))
plt.plot(10**(np.array(Y_test)),color='red')
plt.plot(10**(Y_pred),color='green')
plt.suptitle('Actual Price Vs Predicted Price', fontsize =18)
plt.legend(['Actual', 'Predicted'], fontsize = 16)
plt.xlabel('No. of Test Data', fontsize= 16)
plt.ylabel('Closing Price', fontsize= 16)
plt.grid()

# **Lasso Regression**

The word “LASSO” stands for Least Absolute Shrinkage and Selection Operator.

Lasso regression is a regularization technique. It is used over regression methods for a more accurate prediction. This model uses shrinkage. Shrinkage is where data values are shrunk towards a central point as the mean.

In [ ]:
#Applying Lasso Regression
lasso = Lasso(alpha=0.005, max_iter = 3000)
lasso.fit(X_train, Y_train) 

In [ ]:
# Accuracy of Lasso regression model
round(lasso.score(X_train, Y_train), 4)

In [ ]:
# Prediction of train data
Y_pred_lasso = lasso.predict(X_test)
Y_pred_lasso

In [ ]:
#Converting Y_test data into array
np.array(Y_test)

In [ ]:
# Test Performance of lasso regression model

print("MSE :",round(mean_squared_error(Y_test, Y_pred_lasso), 4))
print("RMSE :",round(math.sqrt(mean_squared_error(Y_test, Y_pred_lasso)),4))
print("MAE :",round(mean_absolute_error(Y_test, Y_pred_lasso),4))
print("MAPE :",round(mean_absolute_percentage_error(Y_test, Y_pred_lasso),4))
print("R2 :",round(r2_score(Y_test, Y_pred_lasso), 4))

In [ ]:
# Actual price vs Predicted price for lasso regression ploting
plt.figure(figsize= (12,5))
plt.plot(10**(np.array(Y_test)),color='red')
plt.plot(10**(Y_pred_lasso),color='green')
plt.suptitle('Actual Price Vs Predicted Price', fontsize =18)
plt.legend(['Predicted', 'Actual'], fontsize = 16)
plt.xlabel('No of Test Data', fontsize= 16)
plt.ylabel('Closing Price', fontsize = 16)
plt.grid()

## ***Cross Validation of Lasso***

Cross validation (CV) is one of the technique used to test the effectiveness of a machine learning models by dividing data into two segments, one used to learn or train a model and the other used to validate the model.If the model performs well over the test data and gives good accuracy, it means the model hasn’t overfitted the training data and can be used for prediction. It is also a re-sampling procedure used to evaluate a model if we have a limited data.

In [ ]:
# Hyper parameter of Tuning For Lasso Regression
lasso_cv=Lasso()
parameters={'alpha':[1e-15,1e-13,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1e-1,1,5,10,20,30,40,45,50,55,60,100]}
lasso_regressor=GridSearchCV(lasso_cv,parameters,scoring='neg_mean_squared_error',cv=3)
lasso_regressor.fit(X_train,Y_train)

In [ ]:
# Finding best parameter
print("The best fit alpha value is found out to be :" ,lasso_regressor.best_params_)
print("using ",lasso_regressor.best_params_, " the negative mean squared error is: ", lasso_regressor.best_score_)

In [ ]:
Y_lasso_pred1=lasso_regressor.predict(X_test)
Y_lasso_pred1

In [ ]:
#Test performance of Lasso after Cross-Validation
print("MSE value is :",round(mean_squared_error(Y_test, Y_lasso_pred1), 4))
print("RMSE value is :",round(math.sqrt(mean_squared_error(Y_test, Y_lasso_pred1)),4))
print("MAE value is :",round(mean_absolute_error(Y_test, Y_lasso_pred1),4))
print("MAPE value is :",round(mean_absolute_percentage_error(Y_test, Y_lasso_pred1),4))
print("r2 score is :",round(r2_score(Y_test, Y_lasso_pred1), 4))

# **Ridge Regression**

Ridge Regression is an adaptation of the popular and widely used linear regression algorithm. It enhances regular linear regression by slightly changing its cost function, which results in less overfit models.

In [ ]:
#Applying Ridge Regression
ridge = Ridge (alpha= 0.1)
ridge.fit(X_train,Y_train)

In [ ]:
# Accuracy of Ridge regression model
round(ridge.score(X_train, Y_train), 4)

In [ ]:
# Prediction of test data
Y_pred_ridge = ridge.predict(X_test)
Y_pred_ridge

In [ ]:
#Converting Y_test data into array
np.array(Y_test)

# **Conclusion**

Write the conclusion here.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***